# Cleaning and preprocessing I

## Merging the two data files

According to the codebook on [Kaggle](https://www.kaggle.com/datasets/thedevastator/mental-health-in-drug-users-during-covid-19), each respondent's identifier is given in the `hash` column of each data file.  First we load the translated data we saved in the ProblemStatement notebook.

In [1]:
# Import pandas to load the data
import pandas as pd

# Load the first data set
data1 = pd.read_csv("data/translated_data1.csv")

# Load the second data set
data2 = pd.read_csv("data/translated_data2.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/translated_data1.csv'

Now we can check how many respondents completed both surveys by looking for `hash` entries that appear in both data files.  

In [ ]:
# Create a list of the hash ids that appear in both data files
common_hash_ids = []

# Append to the list using a loop
for hash_id in data1['hash'].values:
    if (hash_id in data2['hash'].values)==True:
        common_hash_ids.append(hash_id)

# Count the number of entries in the list
len(common_hash_ids)

Based on the output, there are 7090 respondents who completed both surveys.  We shall create a data frame merging the data from both data files for each of those respondents.

In [ ]:
# Create the new data frame
master_data = pd.merge(data1,
                       data2, 
                       how='inner',
                       on="hash")

# Show the new data frame
master_data    

As expected, `master_data` has 7090 rows.

## Renaming the columns in `data1`

Now it would be helpful if the numbered columns from `data1` were replaced with their descriptions.  To do this we can scrape the table of the column descriptions from [the Kaggle page](https://www.kaggle.com/datasets/thedevastator/mental-health-in-drug-users-during-covid-19).

In [ ]:
# Import BeautifulSoup to scrape data 
from bs4 import BeautifulSoup

# Import requests to get the html
import requests

# This is the url for the html we want
kaggle_url = "https://www.kaggle.com/datasets/thedevastator/mental-health-in-drug-users-during-covid-19"

# Request the html code and assign it to a variable
kaggle_html = requests.get(kaggle_url)

# Make a BeautifulSoup object
kaggle_soup = BeautifulSoup(kaggle_html.content)

# Show the html
print(kaggle_soup.prettify())

The table appears twice, in the head element and in the body element.  It happens to occur in the first `script` element in the body.  

In [ ]:
# Finds the html element that contains the table
kaggle_table = kaggle_soup.body.find('script').text # Turns the html into a string
kaggle_table

The table is made using Markdown.  To extract it from the html element, we can use the split function to separate each row of the table by the string `|\\n`.  Note that the original html only had one backslash, but when we turned the html into a string an escape character was added.

In [ ]:
# Use "|\\n" to split the html according to the rows of the table
kaggle_strings = kaggle_table.split("|\\n")
kaggle_strings

Now we have a list of strings that includes each line of the table, but we don't want the first two strings or the last one, so we can get rid of them.

In [ ]:
# Indices of the unwanted strings
unwanted = [0,1,len(kaggle_strings)-1]

# Delete the unwanted strings
for kaggle_strings_index in sorted(unwanted, 
                                   reverse=True): # It's good practice to delete the highest indices first, so we reverse the order of the indices
    del kaggle_strings[kaggle_strings_index]
kaggle_strings

Now that we've isolated the rows of the table we need to separate the two columns.

In [ ]:
# Initialize the column lists
numbered_columns = []
descriptions = []

# Split the strings into two strings, the first goes into one column and the second goes into the other
for line in kaggle_strings:
    split_line = line.split(" | ")
    numbered_columns.append(split_line[0])
    descriptions.append(split_line[1])

# Show the first column    
numbered_columns

In [ ]:
# Show the second column
descriptions

Now we need to make the columns in `numbered_columns` match the columns in `master_data`.  We can use the `split` function again.

In [ ]:
# Rename the entries in numbered_columns
for entry in numbered_columns:
    entry_index = numbered_columns.index(entry)
    split_entry = entry.split("**")
    entry = split_entry[1]
    numbered_columns[entry_index] = entry
numbered_columns    

Now make a loop that will change the column names in `master_data`.

In [ ]:
# Change the names of the columns in master_data
for heading in numbered_columns:
    if heading in master_data.columns.values.tolist():
        heading_index = numbered_columns.index(heading)
        master_data.rename(columns={heading:descriptions[heading_index]},
                          inplace=True)
master_data.columns.values.tolist()        

We've used all the information from Kaggle to make the column names more descriptive, but it's still not perfect.  There are still many numbered columns with no description and many of the columns extracted from `data2` have titles that are not descriptive enough.  The next step is to delete unwanted columns.  This is done in the notebook [CleaningAndPreprocessingII](CleaningAndPreprocessingII.ipynb).

The last step in this notebook is to save the `master_data` to a file to call in the next notebook.

In [ ]:
# Save master_data to a csv file
master_data.to_csv("data/master_data.csv")